# 0. 드라이브 마운트 (★)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. 필요 라이브러리 설치 및 임포트 (★)

In [ ]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install -q transformers sentence-transformers gradio chromadb
!pip install -U bitsandbytes

In [5]:
from PIL import Image
import os
import io
import torch
import numpy as np
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import gradio as gr
import chromadb
from chromadb.config import Settings
import json

# 2. 텍스트/이미지 임베딩 모델 임포트 (★)

In [ ]:
# 텍스트 임베딩 모델
text_embedder =  SentenceTransformer("BAAI/bge-m3")

# 이미지 임베딩 모델
clip_model_name = "openai/clip-vit-base-patch32"
clip_model = CLIPModel.from_pretrained(clip_model_name)
clip_processor = CLIPProcessor.from_pretrained(clip_model_name)

# 3. ChromaDB 불러오기 (★)

In [7]:
# ChromaDB 불러오기
client = chromadb.PersistentClient(path="drive/MyDrive/Movie_DB")

text_collection = client.get_collection("movie_text_embeddings")
image_collection = client.get_collection("movie_image_embeddings")

# 4. ChromaDB 데이터 search 함수 정의 (★)

In [8]:
def search_with_metadata(query_text=None, query_image=None, k=1):
    if query_image: # query에 이미지가 있는 경우,
        inputs = clip_processor(images=query_image, return_tensors="pt")
        with torch.no_grad():
            q_emb = clip_model.get_image_features(**inputs).cpu().numpy().tolist()

        out = image_collection.query(
            query_embeddings=q_emb,
            n_results=k
        )
    else: # query에 텍스트만 있는 경우
        q_emb = text_embedder.encode(query_text).tolist()

        out = text_collection.query(
            query_embeddings=[q_emb],
            n_results=k
        )

    ids = out["ids"][0]
    metas = out["metadatas"][0]

    return {
        "ids": ids,
        "metadatas": metas,
    }

# 5. Q&A 상호작용을 위한 LLM 모델 로드 (★)

In [ ]:
model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# 6. Q&A LLM 파이프라인 함수 정의 (★)

### 6-1. 질의용 전체 파이프라인 함수 (★)

In [10]:
def answer_query(user_text=None, user_image=None):

    # 1. 검색 결과 top-k row index 가져오기
    match_indices = search_with_metadata(query_text=user_text, query_image=user_image, k=1)

    # 2. 영화 정보 context 구성
    meta = match_indices["metadatas"][0]  # top-1 결과
    title = meta.get("title", "")
    genres = meta.get("genres", "")
    overview = meta.get("overview", "")
    popularity = meta.get("popularity", "")
    release_date = meta.get("release_date", "")
    budget = meta.get("budget", "")
    revenue = meta.get("revenue", "")
    runtime = meta.get("runtime", "")
    context = ""

    context = (
        f"title: {title}\n"
        f"genres: {genres}\n"
        f"overview: {overview}\n"
        f"popularity: {popularity}\n"
        f"release_date: {release_date}\n"
        f"budget: {budget}\n"
        f"revenue: {revenue}\n"
        f"runtime: {runtime}\n"
    )

    # 3. 사용자 입력 텍스트 구성
    user_prompt_text = user_text if user_text else "User upload the image."

    # 4. Qwen2용 messages 구성
    messages = [
        {
            "role": "system",
            "content": (
                "You are a movie expert."
                "Answer the user's questions in natural, smooth english only, using the [reference data] as your knowledge source. "
                "The [reference data] contains movie information you should consult to find the most relevant details for your answer."
                "If the movie the user is asking about does not have sufficiently similar information in the [reference data], using your pre-trained knowledge."
                "Always summarize your answer in a single concise sentence."
            )
        },
        {
            "role": "user",
            "content": (
                f"[Question]\n{user_prompt_text}\n\n"
                f"[reference data]\n{context}\n\n"
                "[Answer]"
            )
        }
    ]

    # 5. apply_chat_template로 모델 입력 생성
    inputs = tokenizer.apply_chat_template(
        messages,
    	add_generation_prompt=True,
    	tokenize=True,
    	return_dict=True,
    	return_tensors="pt",
    ).to(model.device)


    # 6. 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
    )

    # 7. 모델 출력 디코딩
    generated = tokenizer.decode(
        outputs[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )
    return generated.strip()

### 6-2. json 파일 생성용 추가 파이프라인 (★)

In [ ]:
INPUT_DIR = "./data"
OUTPUT_DIR = "./release"

# json 파일 로드용 함수
def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

# json 파일 저장용 함수
def save_json(path, data):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

########################################################################
FILENAME = "test.json" # 불러올 입력 데이터 파일 이름
FILENAME_SAVE = "202000826.test.json"
########################################################################

input_path = os.path.join(INPUT_DIR, FILENAME)
data = load_json(input_path)

output = [] # 저장할 Q&A 데이터 쌍

for item in data:
  # json 파일에서 질문 가져오기
  question = item['question']
  print(f"▶ Generating answer for: {question}")

  # 각 질문에 대한 대답 생성
  answer = answer_query(question)
  print(f"▶ Generated answer: {answer}")

  # json 파일로 저장하기 위한 포멧 설정
  output_data = {
      "question": question,
      "answer": answer
  }
  output.append(output_data)

output_path = os.path.join(OUTPUT_DIR, FILENAME_SAVE)
save_json(output_path, output)

print(f"✔ Saved: {output_path}")

#별첨. Gradio 앱 실행

##### 가능한 질의 내용: 제목, 장르, 줄거리, 인기 정도, 출시일, 예산, 수익, 상영시간
### (1) 채팅으로만 질의할 경우
- 텍스트 칸에 영화 관련 질문 작성 후 send

### (2) 이미지(포스터)와 함께 질의할 경우
- 이미지 업로드 후, 텍스트 칸에 질문 작성 후 send
- 예: 포스터 업로드 후, 'Let me know this movie's genres.'


In [ ]:
def chat_fn(query, image):
    return answer_query(user_text=query, user_image=image)

demo = gr.Interface(
    fn=chat_fn,
    inputs=[gr.Textbox(label="질문 입력"), gr.Image(type="pil", label="이미지 업로드")],
    outputs=gr.TextArea(label="챗봇 답변", scale=50),
    title="🎬 영화 포스터 Q&A 챗봇",
)

demo.queue(max_size=20)
demo.launch(share=True)

# 별첨. ChromaDB 생성 코드

### 1. 학습 데이터 불러오기

In [ ]:
from datasets import load_dataset, Dataset
ds = load_dataset("stzhao/movie_posters_100k_controlnet")

### 2. 학습 데이터 전처리: 텍스트 임베딩용 문자열 생성

In [ ]:
def preprocess(sample):
    # genres 문자열로 변환
    genre_names = [g['name'] for g in sample['genres']]
    genres_str = ", ".join(genre_names)

    # 텍스트 임베딩용 문자열
    sample['text_for_embedding'] = f"{sample['title']}\nGenres: {genres_str}\nOverview: {sample['overview']}\nRelease_date: {sample.get('release_date','')}"
    return sample

dataset = ds.map(preprocess)

### 3. ChromaDB 데이터 생성

In [ ]:
import tqdm

# 1. ChromaDB 초기화
client = chromadb.PersistentClient(path="my_chroma_db")

# 텍스트 임베딩 저장 collection
text_collection = client.get_or_create_collection(
    name="movie_text_embeddings",
    metadata={"description": "영화 텍스트 임베딩 저장"}
)

# 이미지 임베딩 저장 collection
image_collection = client.get_or_create_collection(
    name="movie_image_embeddings",
    metadata={"description": "영화 이미지 임베딩 저장"}
)

# 2. Embedding 생성 및 저장
text_embeddings = []
text_metadatas = []
text_documents = []

image_embeddings = []
image_metadatas = []

id_number = list(map(str, range(1, 91001))) # 총 91000개의 데이터 임베딩용 ID 생성

for sample in tqdm(dataset):

    # 텍스트 임베딩
    t_emb = text_embedder.encode(sample["text_for_embedding"])
    t_emb = t_emb.tolist()

    text_embeddings.append(t_emb)
    text_metadatas.append({
        "title": sample["title"],
        "genres": sample["genres"],
        "overview": sample["overview"],
        "release_date": sample["release_date"],
        "budget": sample["budget"],
        "revenue": sample["revenue"],
        "runtime": sample["runtime"],
    })
    text_documents.append(sample["text_for_embedding"])

    # 이미지 임베딩
    image = sample['image']
    inputs = clip_processor(images=image, return_tensors="pt")

    with torch.no_grad():
        i_emb = clip_model.get_image_features(**inputs)

    i_emb = i_emb.cpu().numpy().squeeze().tolist()

    image_embeddings.append(i_emb)
    image_metadatas.append({
        "title": sample["title"],
        "genres": sample["genres"],
        "overview": sample["overview"],
        "release_date": sample["release_date"],
        "budget": sample["budget"],
        "revenue": sample["revenue"],
        "runtime": sample["runtime"],
    })

### 4. ChromaDB 데이터 저장

In [ ]:
def batched_add(collection, ids, embeddings, documents=None, metadatas=None, batch_size=5000):
    n = len(ids)
    for i in range(0, n, batch_size):
        end = i + batch_size
        batch_ids = ids[i:end]
        batch_emb = embeddings[i:end]
        batch_docs = documents[i:end] if documents is not None else None
        batch_meta = metadatas[i:end] if metadatas is not None else None

        collection.add(
            ids=batch_ids,
            embeddings=batch_emb,
            documents=batch_docs,
            metadatas=batch_meta,
        )

# 메타 데이터를 ChromaDB 형식에 맞게 수정 -----
temp = text_metadatas[:]

for item in temp:
    item['genres'] = ", ".join([g["name"] for g in item['genres']])
# ----------------------------------------------

batched_add(
    text_collection,
    ids=id_number,
    embeddings=text_embeddings,
    documents=text_documents,
    metadatas=temp,
)

batched_add(
    image_collection,
    ids=id_number,
    embeddings=image_embeddings,
    metadatas=temp,
)